url : https://www.kaggle.com/rshally/porto-xgb-lgb-kfold-lb-0-282
<br>
study : https://www.youtube.com/watch?v=Fsbx-q3IOek&list=PLC_wC_PMBL5NLKkMooi-n4iK4gv3VqXyo&index=4

다음 강의 전에 스태킹을 알아보고 가자. 정말 많이 사용하는 스킬이다.

This is a kfold with both xgb and lgb. Some elements are from kernels by The 1owl and Hyungdukkang.m,

# Contents

1. [Import Libaray](#import_library)
1. [Import Dataset](#import_data)
1. [Dataset_Preprocess](#dataset_preprocess)
1. [Define function](#define_function)
1. [XGB](#xgb)
1. [LGB](#lgb)

<a class="anchor" id="import_library"></a>
# 1. Import Libarary

In [2]:
import pandas as pd
import numpy as np
import xgboost as xgb
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
import gc

/Users/jang/.pyenv/versions/3.6.5/lib/python3.6/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


- lightgbm 설치 오류시 : https://github.com/Microsoft/LightGBM/issues/1369


    pip uninstall lightgbm
    git clone --recursive https://github.com/Microsoft/LightGBM
    cd LightGBM
    export CXX=g++-8 CC=gcc-8
    mkdir build
    cd build
    cmake ..
    make -j4
    
    pip install --no-binary :all: lightgbm
    
    In [1]: import lightgbm

    In [2]: lightgbm.__version__
    Out[2]: '2.1.1'

<a class="anchor" id="import_data"></a>

# 2. Improt Dataset

너무 데이터가 크니 부분만 불러와 실습해본다.

In [41]:
# 10000줄만 불러온다.
print('loading files...')
train = pd.read_csv('../../input/porto/train.csv', na_values=-1, nrows=10000)
test = pd.read_csv('../../input/porto/test.csv', na_values=-1, nrows=10000)
print(train.shape)
train.head()

loading files...
(10000, 59)


,id,target,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,...,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
0,7,0,2,2.0,5,1.0,0.0,0,1,0,...,9,1,5,8,0,1,1,0,0,1
1,9,0,1,1.0,7,0.0,0.0,0,0,1,...,3,1,1,9,0,1,1,0,1,0
2,13,0,5,4.0,9,1.0,0.0,0,0,1,...,4,2,7,7,0,1,1,0,1,0
3,16,0,0,1.0,2,0.0,0.0,1,0,0,...,2,2,4,9,0,0,0,0,0,0
4,17,0,0,2.0,0,1.0,0.0,1,0,0,...,3,1,1,3,0,0,0,1,1,0


In [42]:
train_org = train
test_org = test

In [43]:
# ps_calc으로 시작하는 칼럼을 제거하기 위해 추출한다.
col_to_drop = train.columns[train.columns.str.startswith('ps_calc_')]
col_to_drop

Index(['ps_calc_01', 'ps_calc_02', 'ps_calc_03', 'ps_calc_04', 'ps_calc_05',
       'ps_calc_06', 'ps_calc_07', 'ps_calc_08', 'ps_calc_09', 'ps_calc_10',
       'ps_calc_11', 'ps_calc_12', 'ps_calc_13', 'ps_calc_14',
       'ps_calc_15_bin', 'ps_calc_16_bin', 'ps_calc_17_bin', 'ps_calc_18_bin',
       'ps_calc_19_bin', 'ps_calc_20_bin'],
      dtype='object')

In [44]:
# 처음 59개 에서 39개로 줄었다.
train = train.drop(col_to_drop, axis=1)
test = test.drop(col_to_drop, axis=1)
print(train.shape)
train.head()

(10000, 39)


,id,target,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,...,ps_car_07_cat,ps_car_08_cat,ps_car_09_cat,ps_car_10_cat,ps_car_11_cat,ps_car_11,ps_car_12,ps_car_13,ps_car_14,ps_car_15
0,7,0,2,2.0,5,1.0,0.0,0,1,0,...,1.0,0,0.0,1,12,2,0.400000,0.883679,0.370810,3.605551
1,9,0,1,1.0,7,0.0,0.0,0,0,1,...,1.0,1,2.0,1,19,3,0.316228,0.618817,0.388716,2.449490
2,13,0,5,4.0,9,1.0,0.0,0,0,1,...,1.0,1,2.0,1,60,1,0.316228,0.641586,0.347275,3.316625
3,16,0,0,1.0,2,0.0,0.0,1,0,0,...,1.0,1,3.0,1,104,1,0.374166,0.542949,0.294958,2.000000
4,17,0,0,2.0,0,1.0,0.0,1,0,0,...,1.0,1,2.0,1,82,3,0.316070,0.565832,0.365103,2.000000


<a class="anchor" id="dataset_preprocess"></a>

# 3. Dataset Preprocess

불러온 데이터의 형식을 맞추어준다. float64 형태를 np.float64로 바꾸어 준다. 마찬가지로 int64도 작업해준다.

In [32]:
train = train_org
test = test_org

In [45]:
train.shape

(10000, 39)

In [46]:
for c in train.select_dtypes(include=['float64']).columns:
    print('%s columns change float64 to np.float64' % c)
    train[c] = train[c].astype(np.float32)
    test[c]= test[c].astype(np.float32)

ps_ind_02_cat columns change float64 to np.float64
ps_ind_04_cat columns change float64 to np.float64
ps_ind_05_cat columns change float64 to np.float64
ps_reg_01 columns change float64 to np.float64
ps_reg_02 columns change float64 to np.float64
ps_reg_03 columns change float64 to np.float64
ps_car_01_cat columns change float64 to np.float64
ps_car_03_cat columns change float64 to np.float64
ps_car_05_cat columns change float64 to np.float64
ps_car_07_cat columns change float64 to np.float64
ps_car_09_cat columns change float64 to np.float64
ps_car_12 columns change float64 to np.float64
ps_car_13 columns change float64 to np.float64
ps_car_14 columns change float64 to np.float64
ps_car_15 columns change float64 to np.float64


In [47]:
# int64를 포함하는 칼럼은 꽤 많다.
train.select_dtypes(include=['int64']).columns[2:]

Index(['ps_ind_01', 'ps_ind_03', 'ps_ind_06_bin', 'ps_ind_07_bin',
       'ps_ind_08_bin', 'ps_ind_09_bin', 'ps_ind_10_bin', 'ps_ind_11_bin',
       'ps_ind_12_bin', 'ps_ind_13_bin', 'ps_ind_14', 'ps_ind_15',
       'ps_ind_16_bin', 'ps_ind_17_bin', 'ps_ind_18_bin', 'ps_car_02_cat',
       'ps_car_04_cat', 'ps_car_06_cat', 'ps_car_08_cat', 'ps_car_10_cat',
       'ps_car_11_cat', 'ps_car_11'],
      dtype='object')

In [48]:
for c in train.select_dtypes(include=['int64']).columns[2:]:
    train[c]= train[c].astype(np.int8)
    test[c] = test[c].astype(np.int8)

In [50]:
# 작업 완료후 확인 출력을 해본다.
# train은 39개 변수, test는 38개 변수를 가진다.
print(train.shape, test.shape)

(10000, 39) (10000, 38)


<a class="anchor" id="define_function"></a>

# 4. Define Function

분석에 사용할 기능들을 정의해줍니다.

In [135]:
def gini(y, pred):
    g = np.asarray(np.c_[y, pred, np.arange(len(y))], dtype=np.float)
    g = g[np.lexsort((g[:,2], -1*g[:,1]))]
    gs = g[:,0].cumsum().sum() / g[:,0].sum()
    gs -= (len(y)) /2
    return gs / len(y)

def gini_xgb(pred, y):
    y = y.get_label()
    return 'gini', gini(y, pred) / gini(y, y)

def gini_lgb(preds, dtrain):
    y = list(dtrain.get_label())
    score = gini(y, preds) / gini(y, y)
    return 'gini', score, True


<a class='anchor' id='xgb'></a>

# 5. XGB


In [138]:
# 파라미터를 정의해준다.
params = {'eta':0.02, 'max_depth':4, 'subsample':0.9, 'colsample_bytree': 0.9 ,
          'objective': 'binary:logistic', 'eval_metric': 'auc', 'silent':True}

In [139]:
X = train.drop(['id', 'target'], axis=1)
features = X.columns
X = X.values
y = train['target'].values
sub=test['id'].to_frame()
sub['target']=0

In [140]:
# drop후 남은 칼럼들
features

Index(['ps_ind_01', 'ps_ind_02_cat', 'ps_ind_03', 'ps_ind_04_cat',
       'ps_ind_05_cat', 'ps_ind_06_bin', 'ps_ind_07_bin', 'ps_ind_08_bin',
       'ps_ind_09_bin', 'ps_ind_10_bin', 'ps_ind_11_bin', 'ps_ind_12_bin',
       'ps_ind_13_bin', 'ps_ind_14', 'ps_ind_15', 'ps_ind_16_bin',
       'ps_ind_17_bin', 'ps_ind_18_bin', 'ps_reg_01', 'ps_reg_02', 'ps_reg_03',
       'ps_car_01_cat', 'ps_car_02_cat', 'ps_car_03_cat', 'ps_car_04_cat',
       'ps_car_05_cat', 'ps_car_06_cat', 'ps_car_07_cat', 'ps_car_08_cat',
       'ps_car_09_cat', 'ps_car_10_cat', 'ps_car_11_cat', 'ps_car_11',
       'ps_car_12', 'ps_car_13', 'ps_car_14', 'ps_car_15'],
      dtype='object')

In [141]:
nrounds = 200   # need to chage to 2000
kfold=2   # need to change to 5
skf = StratifiedKFold(n_splits=kfold, random_state=0)  # StratifiedKFold는 Fold를 균등하게 나누는 역할을 한다
skf

StratifiedKFold(n_splits=2, random_state=0, shuffle=False)

In [142]:
for i, (train_index, test_index) in enumerate(skf.split(X,y)):
    print('xgb_kfold: {} of {} : '.format(i+1, kfold))
    X_train, X_valid = X[train_index], X[test_index]
    y_train, y_valid = y[train_index], y[test_index]
    # 빠른 학습을 위해 DMatrix를 생성
    d_train = xgb.DMatrix(X_train, y_train)
    d_valid = xgb.DMatrix(X_valid, y_valid)
    # 학습이 잘 진행되고 있는지 관찰하기 위함
    watchlist = [(d_train, 'train'), (d_valid, 'valid')]
    # nrounds를 나누어서 중간에 끊어줄 수 있다.
    # nrounds = 200
    xgb_model = xgb.train(params, d_train, nrounds, watchlist, early_stopping_rounds=100, \
                          feval=gini_xgb, maximize=True, verbose_eval=100)    
    sub['target'] += xgb_model.predict(xgb.DMatrix(test[features].values),
                                      ntree_limit=xgb_model.best_ntree_limit+50)/(2*kfold)

xgb_kfold: 1 of 2 : 
[0]	train-auc:0.598547	valid-auc:0.574696	train-gini:0.147291	valid-gini:0.145091
Multiple eval metrics have been passed: 'valid-gini' will be used for early stopping.

Will train until valid-gini hasn't improved in 100 rounds.
[100]	train-auc:0.804349	valid-auc:0.59932	train-gini:0.608787	valid-gini:0.198788
Stopping. Best iteration:
[43]	train-auc:0.745446	valid-auc:0.604874	train-gini:0.490869	valid-gini:0.210697

xgb_kfold: 2 of 2 : 
[0]	train-auc:0.559529	valid-auc:0.51497	train-gini:0.140856	valid-gini:-0.056098
Multiple eval metrics have been passed: 'valid-gini' will be used for early stopping.

Will train until valid-gini hasn't improved in 100 rounds.
[100]	train-auc:0.770529	valid-auc:0.592031	train-gini:0.541348	valid-gini:0.184145
Stopping. Best iteration:
[62]	train-auc:0.743675	valid-auc:0.599006	train-gini:0.486954	valid-gini:0.195976



In [119]:
# 훈련 로그를 보기 위함
xgb.train(params, d_train, nrounds, watchlist, early_stopping_rounds=10)

[0]	train-auc:0.559529	valid-auc:0.51497
Multiple eval metrics have been passed: 'valid-auc' will be used for early stopping.

Will train until valid-auc hasn't improved in 10 rounds.
[1]	train-auc:0.559529	valid-auc:0.51497
[2]	train-auc:0.616154	valid-auc:0.546116
[3]	train-auc:0.627334	valid-auc:0.574463
[4]	train-auc:0.629487	valid-auc:0.579436
[5]	train-auc:0.632114	valid-auc:0.578819
[6]	train-auc:0.634438	valid-auc:0.5778
[7]	train-auc:0.635865	valid-auc:0.576628
[8]	train-auc:0.635881	valid-auc:0.575611
[9]	train-auc:0.637487	valid-auc:0.577216
[10]	train-auc:0.638958	valid-auc:0.576972
[11]	train-auc:0.639695	valid-auc:0.575927
[12]	train-auc:0.639933	valid-auc:0.575298
[13]	train-auc:0.64158	valid-auc:0.574901
[14]	train-auc:0.661166	valid-auc:0.573222
Stopping. Best iteration:
[4]	train-auc:0.629487	valid-auc:0.579436



In [121]:
# 63에서 가장 베스트인 상태를 의미
xgb_model.best_ntree_limit

63

In [ ]:
sub['target'] += xgb_model.predict(
    xgb.DMatrix(test[features].values),
    ntree_limit=xgb_model.best_ntree_limit+50)/(2*kfold)

xgboost를 2개의 폴드로 나눠서 모델 2개로, 0.5 percent를 채운거.

In [111]:
gc.collect()
sub.head(2)

,id,target
0,0,0.182263
1,1,0.172779


<a class='anchor' id='lgb'></a>

# 6. LGB

XGB와 동일한 원리로 적용한다.

In [143]:
parmas = {'metrix':'auc', 'learning_rate':0.01, 'max_depth':10, 'max_bin':10, 'objective':'binary', 
         'feature_fractions':0.8, 'bagging_fraction':0.9, 'bagging_freq':10, 'min_data':500}

In [144]:
skf = StratifiedKFold(n_splits=kfold, random_state=1)
skf

StratifiedKFold(n_splits=2, random_state=1, shuffle=False)

In [147]:
for i, (train_index, test_index) in enumerate(skf.split(X,y)):
    print(' lgb kfold: {} of {}'.format(i+1, kfold))
    X_train, X_eval = X[train_index], X[test_index]
    y_train, y_eval = y[train_index], y[test_index]

    
    lgb_model = lgb.train(params, lgb.Dataset(X_train, label=y_train), nrounds, 
                  lgb.Dataset(X_eval, label=y_eval), verbose_eval=100, 
                  feval=gini_lgb, early_stopping_rounds=100)
    
    
    
    sub['target'] += lgb_model.predict(test[features].values, num_iteration=lgb_model.best_iteration) / (2*kfold)

    

 lgb kfold: 1 of 2
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's auc: 0.598607	valid_0's gini: 0.197382
Early stopping, best iteration is:
[70]	valid_0's auc: 0.601687	valid_0's gini: 0.203539
 lgb kfold: 2 of 2
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's auc: 0.63052	valid_0's gini: 0.26119
[200]	valid_0's auc: 0.6302	valid_0's gini: 0.260553
Did not meet early stopping. Best iteration is:
[170]	valid_0's auc: 0.633178	valid_0's gini: 0.266507


In [149]:
sub.to_csv('sub10.csv', index=False, float_format='%.5f')
gc.collect()
sub.head(2)

,id,target
0,0,0.077791
1,1,0.079438


============================================================================================================